# 3D shape identification algorithm

### Extract the measures from the 3D mesh files
Specify the path to the folder where the 3D files are stored (line 209) <br> Specify the csv file name to store the extracted measures (line 204). This file will be stored in the same folder the files are stored. <br> <br> Please, note that folder path is also included in the measures output csv file. This is to be able to recover the class at a latter stage, assuming that the models are divided by subfolders according to class within the main folder. 

In [ ]:
import numpy as np
import glob
import os
import pandas as pd
from scipy.spatial import ConvexHull, QhullError
from sklearn.decomposition import PCA
from numpy.fft import fft

def load_stl(folder_path):
    # Function to load all STL files and extract vertices (Binary STL only)
    vertices_dict = {}
    for file_path in glob.glob(os.path.join(folder_path, '**/*.stl'), recursive=True):
        print(f"Loading {file_path}...")
        with open(file_path, 'rb') as file:
            data = np.fromfile(file, dtype=np.uint8, count=-1, offset=84)
            data = data.reshape((-1, 50))
            data = data[:, :48]
            data = data.reshape((-1, 4))[:, 1:]
            vertices = data.reshape((-1, 3))
            vertices_dict[file_path] = vertices
    return vertices_dict

def compute_cross_sectional_areas(vertices, axis_index, n_sections=9):  # Modify according to the n_sections desired
    # Compute cross-sectional areas along a specified axis
    min_val = np.min(vertices[:, axis_index])
    max_val = np.max(vertices[:, axis_index])
    step = (max_val - min_val) / n_sections
    areas = []
    for i in range(n_sections):
        val = min_val + i * step
        cross_section = vertices[np.abs(vertices[:, axis_index] - val) < step / 2]
        if len(cross_section) > 2:
            # Projecting vertices to a plane perpendicular to the chosen axis
            proj_vertices = np.delete(cross_section, axis_index, axis=1)
            try:
                hull = ConvexHull(proj_vertices)
                areas.append(hull.area)
            except QhullError:
                areas.append(0)
        else:
            areas.append(0)
    return areas

def perform_pca_and_get_dimensions(vertices):
    pca = PCA(n_components=3)
    pca.fit(vertices)
    transformed = pca.transform(vertices)
    min_vals = np.min(transformed, axis=0)
    max_vals = np.max(transformed, axis=0)
    dimensions = max_vals - min_vals
    return dimensions, pca.explained_variance_ratio_, pca.components_

def calculate_curvature(vertices):
    curvatures = []
    for i in range(0, len(vertices), 3):  # Assuming vertices are in sets of 3 (triangular facets)
        p1, p2, p3 = vertices[i], vertices[i+1], vertices[i+2]
        v1 = p2 - p1
        v2 = p3 - p1

        # Calculate norms
        norm_v1 = np.linalg.norm(v1)
        norm_v2 = np.linalg.norm(v2)

        if norm_v1 > 1e-6 and norm_v2 > 1e-6:  # Check to avoid division by zero
            dot_product = np.dot(v1, v2)
            dot_product = np.clip(dot_product / (norm_v1 * norm_v2), -1.0, 1.0)  # Clipping to avoid invalid values for arccos
            angle = np.arccos(dot_product)
            curvatures.append(angle)
    
    if curvatures:
        return np.mean(curvatures)
    else:
        return 0

def calculate_fourier_descriptors(vertices, num_descriptors=5):
    # Calculating for XY plane
    vertices_2d_xy = vertices[:, :2]
    fourier_result_xy = fft(vertices_2d_xy, axis=0)
    descriptors_xy = np.abs(fourier_result_xy[:num_descriptors]).flatten()

    # Calculating for XZ plane
    vertices_2d_xz = vertices[:, [0, 2]]  # X and Z coordinates
    fourier_result_xz = fft(vertices_2d_xz, axis=0)
    descriptors_xz = np.abs(fourier_result_xz[:num_descriptors]).flatten()

    # Calculating for YZ plane
    vertices_2d_yz = vertices[:, 1:]  # Y and Z coordinates
    fourier_result_yz = fft(vertices_2d_yz, axis=0)
    descriptors_yz = np.abs(fourier_result_yz[:num_descriptors]).flatten()

    # Combine descriptors from all planes
    return np.concatenate([descriptors_xy, descriptors_xz, descriptors_yz])

def calculate_fourier_descriptors_for_cross_section(cross_section, num_descriptors=5):
    # Assuming cross_section is a 2D array of vertices for the cross section
    fourier_result = fft(cross_section, axis=0)
    return np.abs(fourier_result[:num_descriptors]).flatten()

def compute_fourier_descriptors_cross_sections(vertices, axis_index, n_sections=9, num_descriptors=5):  # Modify according to the n_sections desired
    # Similar steps as in compute_cross_sectional_areas but with Fourier descriptors
    min_val = np.min(vertices[:, axis_index])
    max_val = np.max(vertices[:, axis_index])
    step = (max_val - min_val) / n_sections
    descriptors = []
    for i in range(n_sections):
        val = min_val + i * step
        cross_section = vertices[np.abs(vertices[:, axis_index] - val) < step / 2]
        if len(cross_section) > 2:
            # Projecting vertices to a plane perpendicular to the chosen axis
            proj_vertices = np.delete(cross_section, axis_index, axis=1)
            fourier_descriptors = calculate_fourier_descriptors_for_cross_section(proj_vertices, num_descriptors)
            descriptors.extend(fourier_descriptors)
        else:
            descriptors.extend([0] * num_descriptors * 2)  # Assuming 2D Fourier Descriptors
    return descriptors


def calculate_sphericity(volume, surface_area):
    return (np.pi ** (1 / 3) * (6 * volume) ** (2 / 3)) / surface_area

def calculate_roundness(vertices):
    hull = ConvexHull(vertices)
    sphere_radius = np.cbrt((3 * hull.volume) / (4 * np.pi))
    return hull.area / (4 * np.pi * sphere_radius ** 2)

def calculate_compactness_aabb(vertices):
    min_vals = np.min(vertices, axis=0)
    max_vals = np.max(vertices, axis=0)
    dimensions = max_vals - min_vals
    hull = ConvexHull(vertices)
    volume = hull.volume
    return volume / (np.linalg.norm(dimensions) ** 3)

def extract_features(vertices):
    dimensions, variance_ratios, principal_axes = perform_pca_and_get_dimensions(vertices)
    hull = ConvexHull(vertices)
    volume = hull.volume
    surface_area = hull.area
    aspect_ratios = [dimensions[0]/dimensions[1], dimensions[1]/dimensions[2], dimensions[0]/dimensions[2]]
    elongation = max(dimensions) / min(dimensions)
    compactness = volume / (np.linalg.norm(dimensions) ** 3)
    compactness_aabb = calculate_compactness_aabb(vertices)
    eccentricity = np.sqrt(1 - (min(dimensions) / max(dimensions)) ** 2)
    cross_sectional_areas_x = compute_cross_sectional_areas(vertices, axis_index=0)
    cross_sectional_areas_y = compute_cross_sectional_areas(vertices, axis_index=1)
    cross_sectional_areas_z = compute_cross_sectional_areas(vertices, axis_index=2)
    volume_to_surface_area = volume / surface_area
    curvature = calculate_curvature(vertices)
    fourier_descriptors = calculate_fourier_descriptors(vertices)
    fourier_descriptors_x = compute_fourier_descriptors_cross_sections(vertices, axis_index=0)
    fourier_descriptors_y = compute_fourier_descriptors_cross_sections(vertices, axis_index=1)
    fourier_descriptors_z = compute_fourier_descriptors_cross_sections(vertices, axis_index=2)
    sphericity = calculate_sphericity(volume, surface_area)
    roundness = calculate_roundness(vertices)

    return np.array([*dimensions, volume, surface_area, *aspect_ratios, elongation, compactness, compactness_aabb, eccentricity, *cross_sectional_areas_x, *cross_sectional_areas_y, *cross_sectional_areas_z, volume_to_surface_area, *variance_ratios, *principal_axes.flatten(), curvature, *fourier_descriptors, *fourier_descriptors_x, *fourier_descriptors_y, *fourier_descriptors_z, sphericity, roundness])

def process_stl_files(folder_path):
    features = []
    file_info = []
    vertices_dict = load_stl(folder_path)

    for file_path, vertices in vertices_dict.items():
        print(f"Processing {file_path}...")
        model_features = extract_features(vertices)
        features.append(model_features)
        file_name = os.path.basename(file_path)
        folder_name = os.path.dirname(file_path)
        file_info.append((file_name, folder_name))

    return np.array(features), file_info

def export_to_csv(features, file_info, folder_path):
    column_names = [
        'Length', 'Width', 'Height', 'Volume', 'Surface Area',
        'Aspect Ratio 1', 'Aspect Ratio 2', 'Aspect Ratio 3',
        'Elongation', 'Compactness', 'Compactness_aabb', 'Eccentricity',
        'Cross-Sectional Area X1', 'Cross-Sectional Area X2', 'Cross-Sectional Area X3', 'Cross-Sectional Area X4', 'Cross-Sectional Area X5', 'Cross-Sectional Area X6', 'Cross-Sectional Area X7', 'Cross-Sectional Area X8', 'Cross-Sectional Area X9',  # Assuming 5 sections as per n_sections
        'Cross-Sectional Area Y1', 'Cross-Sectional Area Y2', 'Cross-Sectional Area Y3', 'Cross-Sectional Area Y4', 'Cross-Sectional Area Y5', 'Cross-Sectional Area Y6', 'Cross-Sectional Area Y7', 'Cross-Sectional Area Y8', 'Cross-Sectional Area Y9',  # Assuming 5 sections as per n_sections
        'Cross-Sectional Area Z1', 'Cross-Sectional Area Z2', 'Cross-Sectional Area Z3', 'Cross-Sectional Area Z4', 'Cross-Sectional Area Z5', 'Cross-Sectional Area Z6', 'Cross-Sectional Area Z7', 'Cross-Sectional Area Z8', 'Cross-Sectional Area Z9',  # Assuming 5 sections as per n_sections
        'Volume to Surface Area Ratio', 'PCA Variance Ratio 1', 'PCA Variance Ratio 2', 'PCA Variance Ratio 3',
        'PCA Component 1-1', 'PCA Component 1-2', 'PCA Component 1-3',
        'PCA Component 2-1', 'PCA Component 2-2', 'PCA Component 2-3',
        'PCA Component 3-1', 'PCA Component 3-2', 'PCA Component 3-3',
        'Curvature', 'Sphericity', 'Roundness'
    ]

    # Adding column names for Fourier descriptors from XY, XZ, YZ projections
    for plane in ['XY', 'XZ', 'YZ']:
        for descriptor in range(1, 11):  # Assuming 10 descriptors
            column_names.append(f'Fourier Descriptor {plane}-{descriptor}')

    # Adding column names for Fourier descriptors for each cross section
    for axis in ['X', 'Y', 'Z']:
        for section in range(1, 10):  # Assuming 9 sections as per n_sections
            for descriptor in range(1, 11):  # Assuming 10 descriptors
                column_names.append(f'Cross-Section Fourier Descriptor {axis}{section}-{descriptor}')

    df = pd.DataFrame(features, columns=column_names)
    df['File Name'] = [info[0] for info in file_info]
    df['Folder Path'] = [info[1] for info in file_info]
    df['Class'] = ''  # Add an empty 'Class' column for the user to fill in later

    output_path = os.path.join(os.path.dirname(folder_path), 'measures_file.csv')  # Replace with the your own file name
    df.to_csv(output_path, index=False)
    print(f"Data exported to {output_path}")

# Specify the folder path
folder_path = 'C:/3D_models/oriented_SH'  # Replace with the folder path in which the stl files are stored (it will also extract the files from subfolders)
features, file_info = process_stl_files(folder_path)

# Export the measurements to CSV
export_to_csv(features, file_info, folder_path)


### Normalise the values of the cvs
**Do not execute** unless you are going to use a linear classifier. Even when using a linear classifier such as LDA or SVM, it is recommended to test the performance of the classification agaisnt non-normalised values. <br> Specify the path of the required input file (line 30) with the measures extrated before. <br> Specify the path of the output normalised measures to use later (line 31).

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

def normalize_and_round_csv(input_file, output_file):
    """
    Reads a CSV file, normalizes numerical columns to a range of 0 to 1 (excluding all-NaN columns),
    rounds them to 4 decimal places, and saves the result to a new CSV file.
    """
    # Load the CSV file
    df = pd.read_csv(input_file)

    # Select numerical columns and exclude columns with all NaN values
    numerical_cols = df.select_dtypes(include=['float64', 'int64']).columns
    cols_to_normalize = [col for col in numerical_cols if not df[col].isna().all()]

    # Normalize the selected columns
    scaler = MinMaxScaler()
    df[cols_to_normalize] = scaler.fit_transform(df[cols_to_normalize])

    # Round to 4 decimal places
    df[cols_to_normalize] = df[cols_to_normalize].round(4)

    # Save to new CSV file
    df.to_csv(output_file, index=False)
    print(f"Normalized and rounded CSV saved to {output_file}")

# Main
if __name__ == "__main__":
    # Manually set the input and output file paths
    input_file_path = 'C:/3D_models/oriented_SH/measures_file.csv'  # Replace with your input file path
    output_file_path = 'C:/3D_models/oriented_SH/measures_file_norm.csv'  # Replace with your output file path

    normalize_and_round_csv(input_file_path, output_file_path)


### Get information about the structure of your CSV
This will provide some info about the measures extracted that can be useful to evaluate them at a later stage. <br> Provide the path to the csv file with measures you want analyse (line 4).

In [ ]:
import pandas as pd

# Load the CSV file
file_path = 'C:/3D_models/oriented_SH/measures_file.csv'  # Update this with the actual file path
df = pd.read_csv(file_path)

# Get some basic stats on the file
print("Basic File Stats:")
print(df.info())  # This will include the total number of columns
print()

# For each column, display the first and last 5 rows of unique values
for column in df.columns:
    unique_values = pd.unique(df[column])
    print(f'Column: {column}')
    # Check if there are more than 10 unique values to display them as requested
    if len(unique_values) > 10:
        print(f'Unique values (first 20): {unique_values[:20]}')
        print(f'Unique values (last 20): {unique_values[-20:]}')
    else:  # If 10 or fewer unique values, just print them all
        print(f'Unique values: {unique_values}')
    print('-----------------------------------------------------')

# Additional statistical summary for numerical columns
print("Statistical Summary for Numerical Columns:")
print(df.describe())


### Select the column and values within it to create the class values in the csv file 'class' column
Use the information obtained above (towards the end of the report) to classify each unique path into a class for classification. <br> This cell uses the input specified in the previous cell. Please, select an output file in line 33. <br>The use of folder paths here is based on the assumption that the models were distributed in different folders (see notes to the first cell), each with one specific characteristic that defines the class, e.g.: 'Dundee\\6ROW Bere\\6ROW BERE Orkney'. If you have used another way to mark the class, please, specify it in the code below. <br> Please, note that the report only shows the first and last 20 unique paths if there are more paths you will have to include them.

In [ ]:

# Function to update the 'class' column based on selected unique values from a specific column
def update_class_column(df, column_name, class_mapping):
    '''
    Update the 'class' column in the DataFrame based on selected unique values from a specific column.
    
    Parameters:
    df (DataFrame): The DataFrame to be updated.
    column_name (str): The name of the column whose values will be used to populate the 'class' column.
    class_mapping (dict): A dictionary mapping the unique values of the selected column to the class names.
    '''
    df['Class'] = df[column_name].map(class_mapping)
    return df

# Example usage
column_to_use = 'Folder Path'
mapping = {'C:/ProofsML/OrientedMatched_40K_L50\\Dundee\\2ROW British': 'Brit',
           'C:/ProofsML/OrientedMatched_40K_L50\\Dundee\\2ROW Scottish': 'Scot',
           'C:/ProofsML/OrientedMatched_40K_L50\\Dundee\\6ROW Bere\\6ROW BERE Orkney': 'Bere',
           'C:/ProofsML/OrientedMatched_40K_L50\\Dundee\\6ROW Bere\\6ROW BERE Unknown': 'Bere',
           'C:/ProofsML/OrientedMatched_40K_L50\\Dundee\\6ROW Bere\\6ROW BERE Western Isles': 'Bere',
           'C:/ProofsML/OrientedMatched_40K_L50\\Dundee\\6ROW Faro': 'Faro',
           'C:/ProofsML/OrientedMatched_40K_L50\\Dundee\\6ROW Scandinavian': 'Scand',
           'C:/ProofsML/OrientedMatched_40K_L50\\Orkney\\2ROW British': 'Brit',
           'C:/ProofsML/OrientedMatched_40K_L50\\Orkney\\2ROW Scottish': 'Scot',
           'C:/ProofsML/OrientedMatched_40K_L50\\Orkney\\6ROW Bere\\6ROW BERE ORKNEY': 'Bere',
           'C:/ProofsML/OrientedMatched_40K_L50\\Orkney\\6ROW Bere\\6ROW BERE Unknown': 'Bere',
           'C:/ProofsML/OrientedMatched_40K_L50\\Orkney\\6ROW Bere\\6ROW BERE Western Isles': 'Bere',
           'C:/ProofsML/OrientedMatched_40K_L50\\Orkney\\6ROW Scandinavian': 'Scand',}
update_class_column(df, column_to_use, mapping)

# Save the updated DataFrame to a new CSV file
output_file_path = 'C:/3D_models/oriented_SH/measures_file.csv'  # Update this with the actual output file path
df.to_csv(output_file_path, index=False)


### PCA: for lineal classifiers

**Do not execute** unless you are going to use a linear classifier. Even when using a linear classifier such as LDA or SVM, it is recommended to test the performance of the classification against standard values. <br> Specify the path of the required input file (line 7) with the measures extrated before. <br> Specify the number of principal components (line 19 and 23). <br> Specify the path of the output normalised measures to use later (line 29).

In [ ]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import numpy as np

# Load the data
file_path = 'C:/3D_models/oriented_SH/measures_file.csv'  # Replace with your file path
data = pd.read_csv(file_path)

# Isolating the numerical and non-numerical columns
numerical_data = data.select_dtypes(include=[np.number])
non_numerical_data = data.select_dtypes(exclude=[np.number])

# Standardizing the numerical data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(numerical_data)

# Applying PCA
pca = PCA(n_components=30)
principal_components = pca.fit_transform(scaled_data)

# Converting the principal components into a DataFrame
principal_df = pd.DataFrame(data=principal_components, columns=[f'PC{i+1}' for i in range(30)])

# Concatenating the non-numerical data with the principal components
final_df = pd.concat([non_numerical_data, principal_df], axis=1)

# Save the combined data to a new CSV file
final_df.to_csv('C:/3D_models/oriented_SH/measures_file_PCA30.csv', index=False)

# Optional: Print statements
# To view the explained variance ratio
print(pca.explained_variance_ratio_)

# To view the first few rows of the transformed data
print(final_df.head())


### Select, execute, and validate the classifier
Select the csv file with the extracted measures and the classes defined previously (line 16) <br>
Select the percentage of the total data available for testing purposes (from 0 to 1, line 23). All other data will be used to train the classifier. <br>
Select the classifier to use (line 26)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split, cross_val_score, KFold, LeaveOneOut
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

# Load data
df = pd.read_csv('C:/3D_models/oriented_SH/measures_file.csv')

# Preprocess data
X = df.drop(['File Name', 'Folder Path', 'Class'], axis=1).values
y = LabelEncoder().fit_transform(df['Class'].values)  # Convert classes to integers

# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Specify the classifier
classifier_name = 'gbm'  # Options: 'rf', 'gbm', 'svm', 'knn', 'lda', 'fcnn'

if classifier_name == 'fcnn':
    # FCNN setup
    model = Sequential()
    model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(len(np.unique(y)), activation='softmax'))  # Output layer

    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    # Fit the model
    model.fit(X_train, y_train, epochs=100, batch_size=10, verbose=0)

    # Evaluate the model
    y_pred = np.argmax(model.predict(X_test), axis=-1)
    print(f"Classification Report for FCNN:\n", classification_report(y_test, y_pred, digits=4))
    print(f"Confusion Matrix for FCNN:\n", confusion_matrix(y_test, y_pred))

else:
    # Traditional classifiers setup
    classifiers = {
        'rf': RandomForestClassifier(n_estimators=675, random_state=42),
        'gbm': GradientBoostingClassifier(random_state=42),
        'svm': SVC(random_state=42),
        'knn': KNeighborsClassifier(),
        'lda': LinearDiscriminantAnalysis()
    }
    classifier = classifiers[classifier_name]

    # Perform cross-validation
    cv = KFold(n_splits=5, random_state=42, shuffle=True)
    cv_scores = cross_val_score(classifier, X_train, y_train, cv=cv, scoring='accuracy')

    # Train the model
    classifier.fit(X_train, y_train)

    # Predictions and Evaluation
    y_pred = classifier.predict(X_test)
    print(f"Classification Report for {classifier_name}:\n", classification_report(y_test, y_pred, digits=4))
    print(f"Confusion Matrix for {classifier_name}:\n", confusion_matrix(y_test, y_pred))

    # Visualize the results of cross-validation
    plt.figure(figsize=(8, 6))
    plt.plot(range(1, len(cv_scores) + 1), cv_scores, marker='o', linestyle='--', color='b')
    plt.title(f'Cross-Validation Accuracy Scores for {classifier_name}')
    plt.xlabel('Fold Number')
    plt.ylabel('Accuracy')
    plt.xticks(range(1, len(cv_scores) + 1))
    plt.show()


# Alternative LOO Cros-validation
This cell copies the same process than the previous but using a leave-one-out cross validation method (more computationally costly but more accurate) to test the performance of the classification. It requires the same inputs: <br>
Select the csv file with the extracted measures and the classes defined previously (line 12) <br>
Select the classifier to use (line 20)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import LeaveOneOut, cross_val_predict
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder

# Load your dataframe
df = pd.read_csv('C:/3D_models/oriented_SH/measures_file.csv')

# Preprocess data
le = LabelEncoder()
y = le.fit_transform(df['Class'].values)  # Convert classes to integers
X = df.drop(['File Name', 'Folder Path', 'Class'], axis=1).values

# Specify the classifier
classifier_name = 'gbm'  # Options: 'rf', 'gbm', 'svm', 'knn', 'lda'

# Setup classifiers
classifiers = {
    'rf': RandomForestClassifier(n_estimators=150, random_state=42),
    'gbm': GradientBoostingClassifier(random_state=42),
    'svm': SVC(random_state=42),
    'knn': KNeighborsClassifier(),
    'lda': LinearDiscriminantAnalysis()
}

classifier = classifiers[classifier_name]

# Initialize LeaveOneOut
loo = LeaveOneOut()

# Use cross_val_predict to make predictions for each leave-one-out split
y_pred = cross_val_predict(classifier, X, y, cv=loo, n_jobs=-1)

# Compute confusion matrix
conf_matrix = confusion_matrix(y, y_pred)
print(f"Overall LOOCV Confusion Matrix for {classifier_name}:\n{conf_matrix}")

# Compute classification report
class_report = classification_report(y, y_pred, digits=4, zero_division=0)
print(f"Classification Report for {classifier_name}:\n{class_report}")

# Compute per-class accuracies
class_names = le.classes_
class_accuracies = conf_matrix.diagonal() / conf_matrix.sum(axis=1)
for idx, acc in enumerate(class_accuracies):
    print(f"Accuracy for class '{class_names[idx]}': {acc:.4f}")


## Find the best number of trees

This cell provides a method to estimate the optimal number of trees to employ in a Random Forest classification <br> As in the previous cells, specify the path to the csv file containing the measurements and classes.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import LeaveOneOut, cross_val_predict
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

# Load your data
# Make sure to replace the path with the correct one for your dataset
df = pd.read_csv('C:/3D_models/oriented_SH/measures_file.csv')

# Preprocess data
X = df.drop(['File Name', 'Folder Path', 'Class'], axis=1).values
y = LabelEncoder().fit_transform(df['Class'].values)  # Convert classes to integers

# Define the range of number of trees to test
min_trees = 50
max_trees = 1050
step_size = 50

# Initialize results storage
results = {'Number of Trees': [], 'Accuracy': []}

# Setup LeaveOneOut for cross-validation
loo = LeaveOneOut()

for n_trees in range(min_trees, max_trees + 1, step_size):
    classifier = RandomForestClassifier(n_estimators=n_trees, random_state=42)
    
    # Use cross_val_predict to make predictions for each LOOCV split
    y_pred = cross_val_predict(classifier, X, y, cv=loo, n_jobs=-1)
    
    # Compute accuracy
    accuracy = accuracy_score(y, y_pred)
    
    # Store results
    results['Number of Trees'].append(n_trees)
    results['Accuracy'].append(accuracy)
    print(f"Done for {n_trees} trees: Accuracy = {accuracy}")

# Plot the results
plt.figure(figsize=(10, 6))
plt.plot(results['Number of Trees'], results['Accuracy'], marker='o')
plt.title('LOOCV Accuracy vs. Number of Trees in Random Forest')
plt.xlabel('Number of Trees')
plt.ylabel('Accuracy')
plt.grid(True)
plt.show()